In [17]:
import os
import io
import time
import contextlib
import unicodedata
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras, feature_column
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import PIL.Image as Image
import tensorflow_datasets as tfds

from IPython.core.interactiveshell import InteractiveShell

In [2]:
# 配置项
# 这个要放到设置中文之前否则还是小方框
plt.style.use("seaborn")

# 指定默认字体 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [3]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

2646016/2638744 [==============================] - 1s 0us/step


In [4]:
path_to_file

'/Users/JQC/.keras/datasets/spa-eng/spa.txt'

In [6]:
# 英语和西班牙语
pd.read_table(path_to_file)[:10]

,Go.,Ve.
0,Go.,Vete.
1,Go.,Vaya.
2,Go.,Váyase.
3,Hi.,Hola.
4,Run!,¡Corre!
5,Run.,Corred.
6,Who?,¿Quién?
7,Fire!,¡Fuego!
8,Fire!,¡Incendio!
9,Fire!,¡Disparad!


In [10]:
s = 'bla. bla? bla.bla! bla...'
s = re.sub('([.,!?()])', r' \1 ', s)
s = re.sub('\s{2,}', ' ', s)
s


'bla . bla ? bla . bla ! bla . . . '

In [13]:
# 将Unicode文本转化为ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # 在单词和标点后面添加空格 具体如上面的cell所示 
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    # 用空格替换所有的东东 除了(a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    
    # 在句子前后添加标识符 方便模型预测
    w = '<start> ' + w + ' <end>'
    return w
    

In [14]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [33]:
def create_dataset(path, num_examples):
    """
    清洗句子 删除句子中的特殊字符
    """
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    return zip(*word_pairs)


In [34]:
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [45]:
lines = io.open(path_to_file, encoding='UTF-8').read().strip().split('\n')
word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:10]]
word_pairs
zip(*word_pairs)

[['<start> go . <end>', '<start> ve . <end>'],
 ['<start> go . <end>', '<start> vete . <end>'],
 ['<start> go . <end>', '<start> vaya . <end>'],
 ['<start> go . <end>', '<start> vayase . <end>'],
 ['<start> hi . <end>', '<start> hola . <end>'],
 ['<start> run ! <end>', '<start> corre ! <end>'],
 ['<start> run . <end>', '<start> corred . <end>'],
 ['<start> who ? <end>', '<start> ¿ quien ? <end>'],
 ['<start> fire ! <end>', '<start> fuego ! <end>'],
 ['<start> fire ! <end>', '<start> incendio ! <end>']]

In [36]:
# def max_length(tensor):
#     return max([len(t) for t in tensor])

def max_length(tensor):
    """
    返回最长tensor的长度
    """
    return max(len(t) for t in tensor)

In [50]:
def my_tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    # 以单词列表更新词汇 在调用texts_to_sequences or texts_to_matrix前需要调用fit_on_texts
    # fit_on_sequences 以一系列的序列更新词汇 A "sequence" is a list of integer word indices.
    #                  Required before using sequences_to_matrix (if fit_on_texts was never called).

    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer
    

In [44]:
"""
keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    document_count=0,
    **kwargs,
)
"""
texts = ["你好 我好 你好 你好 你好 我们 大家 都 好 吗 吗 吗 吗 吗", "分词器 训练 文档 训练 文档 文档 你好 我好"]
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)

# fre = tokenizer.word_counts  # 统计词频
# print("type(fre):\n",type(fre))
# print("fre:\n",fre)
# 统计词频
print('【word_counts】: \n', tokenizer.word_counts)

print('【word_docs】: \n', tokenizer.word_docs)

print('【word_index】: \n', tokenizer.word_index)


# 使用dir查看tokenizer的属性和方法
# dir(tokenizer)

print('【word_index】: \n', tokenizer.texts_to_sequences())


'\nkeras.preprocessing.text.Tokenizer(\n    num_words=None,\n    filters=\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\',\n    lower=True,\n    split=\' \',\n    char_level=False,\n    oov_token=None,\n    document_count=0,\n    **kwargs,\n)\n'

【word_counts】: 
 OrderedDict([('你好', 5), ('我好', 2), ('我们', 1), ('大家', 1), ('都', 1), ('好', 1), ('吗', 5), ('分词器', 1), ('训练', 2), ('文档', 3)])
【word_docs】: 
 defaultdict(<class 'int'>, {'大家': 1, '我们': 1, '好': 1, '我好': 2, '你好': 2, '吗': 1, '都': 1, '训练': 1, '分词器': 1, '文档': 1})
【word_index】: 
 {'你好': 1, '吗': 2, '文档': 3, '我好': 4, '训练': 5, '我们': 6, '大家': 7, '都': 8, '好': 9, '分词器': 10}


In [51]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)
    inp_tensor, inp_lang_tokenizer = my_tokenize(inp_lang)
    tar_tensor, tar_lang_tokenizer = my_tokenize(targ_lang)
    return inp_tensor, inp_lang_tokenizer, tar_tensor, tar_lang_tokenizer


In [52]:
# Limit the size of the dataset to experiment faster (optional)
# 限制数据集的大小以提高速度
num_examples = 30000
input_tensor, input_lang_tokenizer, target_tensor, target_lang_tokenizer = load_dataset(path_to_file, 
                                                                                        num_examples=num_examples)
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)


In [53]:
# 划分训练集和测试集
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = model_selection.train_test_split(input_tensor, 
                                                                                                                target_tensor, 
                                                                                                                test_size=0.2)

In [54]:
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))


24000 24000 6000 6000


In [57]:
input_tensor_train

array([[   1,    6,   23, ...,    0,    0,    0],
       [   1,    6,   29, ...,    0,    0,    0],
       [   1,   25,   17, ...,    0,    0,    0],
       ...,
       [   1,   54,   17, ...,    0,    0,    0],
       [   1, 1377,   11, ...,    0,    0,    0],
       [   1, 2291,   10, ...,    0,    0,    0]], dtype=int32)

In [58]:
target_tensor_train

array([[  1,  96,   4, ...,   0,   0,   0],
       [  1,  22,   6, ...,   0,   0,   0],
       [  1,  27, 274, ...,   0,   0,   0],
       ...,
       [  1,  28, 398, ...,   0,   0,   0],
       [  1,   4, 290, ...,   0,   0,   0],
       [  1, 116,  17, ...,   0,   0,   0]], dtype=int32)

In [61]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print('%d ----> %s' % (t, lang.index_word[t]))


In [63]:
print ("Input Language; index to word mapping")
convert(input_lang_tokenizer, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(target_lang_tokenizer, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
6 ----> ¿
23 ----> te
38 ----> puedo
616 ----> llamar
14 ----> de
395 ----> vuelta
5 ----> ?
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
96 ----> may
4 ----> i
122 ----> call
6 ----> you
91 ----> back
7 ----> ?
2 ----> <end>


In [73]:
# 创建Create a tf.data dataset的数据集
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64

steps_per_epoch = len(input_tensor_train) // BATCH_SIZE
embedding_dim = 256
units = 1024

vocab_input_size = len(input_lang_tokenizer.word_index) + 1
vocab_target_size = len(target_lang_tokenizer.word_index) + 1

# print('【target_lang_tokenizer】: \n', target_lang_tokenizer.word_index)
vocab_input_size

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


9414

In [67]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 11]))

In [68]:
# 编码解码模型
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [74]:
encoder = Encoder(vocab_input_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 16, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [75]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [76]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 16, 1)


In [77]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [79]:
decoder = Decoder(vocab_target_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))


Decoder output shape: (batch_size, vocab size) (64, 4935)


In [80]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [81]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [84]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [85]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.5783
Epoch 1 Batch 100 Loss 2.1249
Epoch 1 Batch 200 Loss 1.7670


KeyboardInterrupt: 

In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
translate(u'hace mucho frio aqui.')

In [ ]:
translate(u'esta es mi vida.')

In [ ]:
# wrong translation
translate(u'trata de averiguarlo.')